In [7]:
import numpy 
import scipy


In [10]:
class neuralNetwork:
    #import numpy
    #initialise the network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        import numpy
        import scipy
        
        #set self, inputnodes, hiddennodes, outputnodes, learningrate
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.lr = learningrate
        
        
        #link weight matrices, wih and who
        #weight inside the arrays are w_i_j, where link is from node i to node j in next layer
        #w11 w21
        #w12 w22 etc
        #below is input to hidden
        self.wih = (numpy.random.rand(self.hnodes, self.inodes)-0.5)
        #below is hidden to output
        self.who = (numpy.random.rand(self.onodes, self.hnodes)-0.5)
        
        #a more sophisticated approach, sample the weights from a normal 
        #probability distribution centred around zero and with a standard
        #deviation that is related to the number of incoming links into a 
        #node, 1/√(number of incoming links)
        # (distribution center, std-dev, np-array-size)
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -.5), (self.onodes, self.hnodes))
        
        #activation function is a sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
        
    
    #train network
    def train(self, inputs_list, targets_list):
        #convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        #calculate the signal to hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate signal coming out of hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #calculate signals coming out of final output layer
        final_outputs = self.activataion_function(final_inputs)
        
        #output layer error is the (target-actual)
        output_errors = targets - final_outputs
        #hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        #update the weights for the links between the hidden and output layers
        self.who += self.lr*numpy.dot((output_errors*final_outputs*(1.0-final_outputs)), numpy.transpose(hidden_outputs))
        
        #update the weight for the links between the input and hidden layers
        self.wih += self.lr*numpy.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)), numpy.transpose(inputs))
        
        pass
    
    #QUERY the network
    def query(self, inputs_list):
        #convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate the signals emerging from hiddenlayer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    
    def printThis():
        print(inodes, hnodes, onodes, lr)

In [11]:
inputnodes=3
hiddennodes=3 
outputnodes=3 
learningrate=.8
NN = neuralNetwork(inputnodes, hiddennodes, outputnodes, learningrate)


In [19]:
NN.query([1.0, 3.5, -1.5])

array([[0.56911411],
       [0.5540493 ],
       [0.59691407]])

In [39]:
import numpy as np
# np.random.rand(3,2) -.5
np.random.normal(0.0, pow(9,-0.5),(5,4))

array([[-0.06759488,  0.06715098, -0.23400487, -0.00836146],
       [ 0.3008511 , -0.06956823,  0.05688468, -0.01805967],
       [ 0.30087886, -0.56002772, -0.34785235,  0.79300105],
       [ 0.12334526,  0.0858434 , -0.28231527,  0.59611638],
       [ 0.05656274,  0.08419643, -0.42886879,  0.19947863]])

In [ ]:
#scipy.special for sigmoid function expit()
import scipy.special
#activation func
self.activate_function = lambda x: scipy.expit(x)